In [ ]:
%load_ext autoreload
from google.auth import default
credentials, project = default()

In [ ]:

%autoreload 2
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.config_loader import config
from src.shop import generate_product_images, generate_shopify_csv
from src.zip import create_m4a_zip_collections
config.TARGET_LANGUAGE_NAME

In [3]:

PRODUCT_CONFIG = {
    "individual": {
        "price": 2.99,
        "template": "shop/individual_pack_template.html",
        "indices": list(range(1, 3)),  # or your desired range
    },
    "bundle": {
        "price": 5.99,
        "template": "shop/bundle_pack_template.html",
        "ranges": [[3, 4]],
    },
    "complete": {
        "price": 19.99,
        "template": "shop/complete_pack_template.html",
    }
}

In [ ]:
zips = create_m4a_zip_collections(
    product_config=PRODUCT_CONFIG,
    collection="LM1000",
    bucket_name=config.GCS_PRIVATE_BUCKET,
    output_base_dir="../outputs/gcs",
    use_local_files=True
)

In [ ]:
# Default configuration (stories 1-8, 9-14, 15-20)
print("=== Generating Product Images ===")
generated_images = generate_product_images(product_config=PRODUCT_CONFIG,
    collection="LM1000", generate_individual=True
)

print("\nGenerated images:")
for product_type, uri in generated_images.items():
    print(f"  {product_type}: {uri}")


In [15]:

# 2. Generate the Shopify CSV with default settings
print("\n=== Generating Shopify CSV ===")

csv_path = generate_shopify_csv(
    product_config=PRODUCT_CONFIG,
    collection="LM1000",
    free_individual_count=2
)

print(f"CSV generated at: {csv_path}")




=== Generating Shopify CSV ===
CSV generated at: ../outputs/shopify\swedish_lm1000_shopify.csv
